### 02_Prompting Best Practices with Amazon Nova Models

The effectiveness of prompts is contingent upon the quality of the information provided and the craftsmanship of the prompt itself. Prompts may encompass instructions, questions, contextual details, inputs, and examples to effectively guide the model and enhance the quality of the results. This document outlines strategies and tactics for optimizing the performance of Amazon Nova Family of Models. The methods presented herein may be employed in various combinations to amplify their effectiveness. We encourage users to engage in experimentation to identify the approaches most suitable for their specific needs. More details on prompt best practices can be found [here](https://docs.aws.amazon.com/nova/latest/userguide/prompting-text-understanding.html)


#### Before Starting Prompt Engineering

Before starting prompt engineering, it is highly recommended to have following elements in place, so you can iteratively develop the most optimal prompt for your use case:

1. **Define your use case**: Define your use case you want to achieve on 4 dimensions
    1. What is the Task - Define the task you want to achieve from the model
    2. Whats the Role - Define the role model should act like to accomplish that task
    3. Whats the Response Style - Define the response structure or style that should be followed based on the consumer of the output. 
    4. What set of Instructions to be followed:  Define the  set of instructions that model should follow to respond as per the success criteria
2. **Success Criteria**: Clearly define the success criteria or evaluation criteria. This can be in the form of a list of bullet points or as specific as some evaluation metrics (Eg: Length checks, BLEU Score, Rouge, Format, Factuality, Faithfulness). The success criteria should clearly reflect the definition of "good" that will determine the success of the use case.
3. **Draft Prompt**: Finally, a draft prompt is necessary to initiate the iterative process of prompt engineering.



In [1]:
%store -r

In [2]:
from IPython.display import display, Markdown, Latex
import base64
import boto3
import json

client = boto3.client("bedrock-runtime", region_name="us-east-1")

def call_nova(model, 
              messages, 
              system_message='',
              stream='off',
              max_tokens=512, 
              temp=0.7, 
              top_p=0.99, 
              top_k=20,
              tools=None,
              stop_sequences=[],
             verbose=False):
    system_list = [{"text": system_message}]
    inf_params = {"max_new_tokens": max_tokens, "top_p": top_p, "top_k": top_k, "temperature": temp, "stopSequences":stop_sequences}
    request_body = {"messages": messages, 
                    "system": system_list, 
                    "inferenceConfig": inf_params
                   }
    if tools is not None:
        tool_config=[]
        for tool in tools:
            tool_config.append({"toolSpec": tool})
        request_body['toolConfig'] = {"tools":tool_config}
    if verbose:
        print("Request Body", request_body)
    if stream=='off':
        response = client.invoke_model(modelId=model, body=json.dumps(request_body))
        model_response = json.loads(response["body"].read())
        return model_response, model_response["output"]["message"]["content"][0]['text']
    else:
        response = client.invoke_model_with_response_stream(modelId=model, body=json.dumps(request_body))
        return response["body"]

def get_base64_encoded_value(media_path):
    with open(media_path, "rb") as media_file:
        binary_data = media_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string

def print_output(content_text):
    display(Markdown(content_text))

def validate_json(json_string):
    try:
        # Attempt to parse the JSON string
        parsed_json = json.loads(json_string)
        
        # If successful, return the parsed JSON
        print("Valid JSON")
        return parsed_json
    
    except json.JSONDecodeError as e:
        # If parsing fails, print an error message
        print(f"Invalid JSON: {e}")
        
        # Optionally, you can print the location of the error
        print(f"Error at line {e.lineno}, column {e.colno}")
        
        # Return None to indicate failure
        return None

### Structured Outputs

Sometimes it is essential to make sure that the model only responds in a specific output schema that works best for the downstream use cases (for example, automated workflows where inputs and outputs must always be in a specific format). Amazon Nova models can be instructed to generate responses in a structured way. For example, if the downstream parser expects certain naming of keys in the JSON object, specifying them in an output schema field in your query yields the model to respect that schema. If the need is to be able to parse the schema directly without any preamble, the model can be instructed to output only JSON by saying “Please generate only the JSON output. DO NOT provide any preamble.” at the end of your query. 

#### Using Prefill to Help the Model Get Started

An alternate technique to achieve this efficiently is to nudge the model response via prefilling the assistant content. This technique enables the user to direct the model's actions (putting words in the model's mouth), bypass preambles, and enforce specific output formats such as JSON or XML. For example, by prefilling assistant content with “{” or ```json, you can guide model to skip generating any preamble text and start generating JSON object right away. 


➡️ If the user is explicitly looking for extracting JSON, one common observed pattern is to prefill it with ``json and add a stop sequence on ```, this ensures that the model outputs a JSON object that can be programmatically parsed.


In [3]:
unoptimized_prompt = """Provide details about the best selling full-frame cameras in past three years.
Answer in JSON format with keys like name, brand, price and a summary.
"""

messages = [
        {"role": "user", "content": [{"text": unoptimized_prompt}]},
    ]

In [4]:
model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print_output("--------")
print_output(content_text)
print_output("--------")


[Response Content Text]


--------

{
    "rows": [
        {
            "Name": "Sony A7 IV",
            "Brand": "Sony",
            "Price": "$2,500",
            "Summary": "The Sony A7 IV is a full-frame mirrorless camera that has been very popular in the past three years. It features a 33-megapixel back-illuminated Exmor R CMOS sensor, 10 fps continuous shooting speed, and 4K video recording. It also has an advanced autofocus system with 759 phase-detection points, making it great for both stills and video."
        },
        {
            "Name": "Canon EOS R5",
            "Brand": "Canon",
            "Price": "$3,900",
            "Summary": "The Canon EOS R5 is another popular full-frame mirrorless camera that has been very well-received in the past three years. It features a 45-megapixel full-frame CMOS sensor, 8K video recording, and 12 fps continuous shooting speed with the electronic shutter. It also has an advanced autofocus system with 1053 dual-pixel CMOS AF II points, making it great for both stills and video."
        },
        {
            "Name": "Nikon Z7 II",
            "Brand": "Nikon",
            "Price": "$3,000",
            "Summary": "The Nikon Z7 II is a full-frame mirrorless camera that has been very popular in the past three years. It features a 45.7-megapixel back-illuminated CMOS sensor, 10 fps continuous shooting speed, and 4K video recording. It also has an advanced autofocus system with 493 points, making it great for both stills and video."
        }
    ]
}

--------

### Lets add more schema defination with the right data types and use Prefill

In [5]:
optimized_prompt = """Provide 5 examples of the best selling full-frame cameras in past three years.
Follow the Output Schema as described below:
Output Schema:
{
"name" : <string, the name of product>,
"brand" : <string, the name of product>,
"price" : <integer price>,
"summary": <string, the product summary>
}
Only Respond in Valid JSON, without Markdown
"""
messages = [
        {"role": "user", "content": [{"text": optimized_prompt}]},
        {"role": "assistant", "content": [{"text": "```json"}]},
    ]

model_response, content_text = call_nova(LITE_MODEL_ID,
                                         messages,
                                         stop_sequences=["]"])
print("\n[Response Content Text]")
print("--------")
print(content_text)
print("--------")

print("Testing valid JSON:")
parsed_json = validate_json(content_text)
if parsed_json:
    print(parsed_json)



[Response Content Text]
--------
[
    {
        "name": "Sony Alpha a7 IV",
        "brand": "Sony",
        "price": 2500,
        "summary": "The Sony Alpha a7 IV is a full-frame mirrorless camera that features a 33-megapixel sensor, 10fps shooting, and 759 phase-detection autofocus points."
    },
    {
        "name": "Nikon Z 7II",
        "brand": "Nikon",
        "price": 2999,
        "summary": "The Nikon Z 7II is a full-frame mirrorless camera that offers a 45.7-megapixel sensor, 4K UHD video recording, and dual Expeed 6 image processors."
    },
    {
        "name": "Canon EOS R6",
        "brand": "Canon",
        "price": 2499,
        "summary": "The Canon EOS R6 is a full-frame mirrorless camera that boasts a 20.1-megapixel sensor, 12fps shooting, and advanced autofocus capabilities."
    },
    {
        "name": "Fujifilm GFX 100S",
        "brand": "Fujifilm",
        "price": 10000,
        "summary": "The Fujifilm GFX 100S is a medium-format mirrorless camera feat

### Few Shot Example

Including a few examples of the task within your prompt can help to guide Amazon Nova models to generate responses more aligned with your desired outcome. This technique of providing examples to the model to achieve the desired outcome is called few shot prompting. By including the examples using a structured template, you can enable the models to follow instructions, reduce ambiguity, and enhance the accuracy and quality more reliably. This method also helps in clarifying complex instructions or tasks, making it easier for the models to understand and interpret what is being asked. 

**How adding examples to the prompt help**:
Adding examples can help the model with producing 

* Consistent responses which are uniform to the style of the examples 
* Performant responses due to reducing the chance of misinterpreting instructions, and minimizing hallucinations


**Characteristics of Good Shots in prompt**:
The amount by which model performance improves using few shot prompting will depend on the quality and diversity of your chosen examples. 

* **Select diverse examples**: The examples chosen should represent the distribution of your expected input/output in terms of diversity (ranging from common use cases to edge cases) to adequately cover relevant use cases. It is important to avoid any biases in your examples, as bias in the inputs can cause outputs to be biased as well.
* **Match complexity levels**: The complexity of the examples provided should align with the target task or scenario. It is important to make sure the complexity grade is mapped between expected the input and the chosen example in the prompt.
* **Ensure relevance**: The examples selected should be directly relevant to the problem or objective at hand. This ensures consistency and uniformity in responses. 

➡️ Tip: If the above suggestions not work, it is also recommended to build a RAG-based system that augments the prompt with dynamic selection of shots based on the similarities between a user-input query and an available pool of shots.


In [6]:
no_shot = """Your task is to Classify the following texts into the appropriate snetiment classes  The categories to classify are: 

Sentiment Classes:
- Positive
- Negative
-Neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline. 

"""

messages = [
        {"role": "user", "content": [{"text": no_shot}]}
]

model_response, content_text = call_nova(LITE_MODEL_ID,
                                         messages)
print("\n[Response Content Text]")
print("--------")
print(content_text)
print("--------")


[Response Content Text]
--------
To classify the given text, I will analyze the sentiment expressed in the text.

Text: "The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline."

Step 1: Identify positive or negative words
In this text, there are no explicitly positive or negative words.

Step 2: Analyze the overall sentiment
The text states that the movie makes users think about their lives with the teenagers, which could be considered a positive aspect. However, it also mentions that the audience is unclear on the storyline, which is a negative aspect.

Step 3: Determine the sentiment class
Since the text contains both positive and negative aspects and doesn't lean strongly in either direction, it can be considered as Neutral.

Output: Neutral
--------


As you can see there is too much explaination not even asked for, this makes parsing a bit tricky 

Now lets try adding four shots that conveys the meaning and also forces a more stylistic edit on the response 

### With adding few shot prompting

In [7]:
four_shot = """Your task is to Classify the following texts into the appropriate sentiment classes  The categories to classify are: 

Sentiment Classes:
- Positive
- Negative
- Neutral
Please refer to some examples mentioned below
## Examples:
### Example 1 
Input: The movie was crazy good! I loved it
Output: Positive
Explaination: The text said "good" and "loved" so its positive

### Example 2 
Input: The movie was scary and I got scared!
Output: Neutral
Explaination: The text said "scary" and "scared" which can be both positive and negative depending on people who like scary movies or one who hate


### Example 3
Input: The movie was pathetic not worth the time or money!
Output: Negative
Explaination: The text said "pathetic" and "not worth" which is negative sentiment 

### Example 4
Input: The movie had some plots which were interesting and great while there were some gaps which needed more drama!
Output: Neutral
Explaination: The text said "interesting and great" and "some gaps" making it a mixed opinion hence neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline. 
"""


messages = [
        {"role": "user", "content": [{"text": four_shot}]}
]

model_response, content_text = call_nova(LITE_MODEL_ID,
                                         messages)
print("\n[Response Content Text]")
print("--------")
print(content_text)
print("--------")


[Response Content Text]
--------
Output: Neutral

Explanation: The text mentions that the movie "makes users think about their lives" which is a positive aspect, but it also says the "audience unclear on the storyline" which is a negative aspect. Since the text contains both positive and negative elements, it is classified as neutral.
--------


In [8]:
no_cot = """You are a project manager for a small software development team tasked with launching a new app feature. 
You want to streamline the development process and ensure timely delivery. Draft a project plan
"""
messages = [
        {"role": "user", "content": [{"text": no_cot}]}
]

model_response, content_text = call_nova(LITE_MODEL_ID,
                                         messages, max_tokens=1024)
print("\n[Response Content Text]")
print_output("--------")
print_output(content_text)
print_output("--------")



[Response Content Text]


--------

# Project Plan for New App Feature Launch

## 1. Project Overview
- **Project Name**: New App Feature Development
- **Project Start Date**: [Insert Start Date]
- **Project End Date**: [Insert End Date]
- **Project Manager**: [Your Name]
- **Team Members**: [List of Team Members]
- **Objective**: To develop and launch a new app feature by the end of the project period, ensuring it meets quality standards and user requirements.

## 2. Project Scope
- **Feature Description**: A brief description of the new feature.
- **In-Scope**: 
  - Frontend design and implementation
  - Backend development
  - Integration with existing app
  - Basic testing and debugging
- **Out-of-Scope**: 
  - Extensive marketing campaigns
  - User onboarding materials (beyond basic documentation)
  - Post-launch support unless critical bugs are found

## 3. Milestones
- **Milestone 1: Project Initiation**
  - Date: [Insert Date]
  - Deliverables: Project Plan, Requirements Documentation, Initial Team Meeting

- **Milestone 2: Design Phase Completion**
  - Date: [Insert Date]
  - Deliverables: Wireframes, Design Mockups, User Flow Diagrams

- **Milestone 3: Development Phase Completion**
  - Date: [Insert Date]
  - Deliverables: Code Base, Integrated Feature, Initial Testing

- **Milestone 4: Testing Phase Completion**
  - Date: [Insert Date]
  - Deliverables: Test Reports, Bug Fixes, User Acceptance Testing (UAT) Completion

- **Milestone 5: Launch Preparation**
  - Date: [Insert Date]
  - Deliverables: Final Deployment Package, Launch Checklist

- **Milestone 6: Post-Launch Review**
  - Date: [Insert Date]
  - Deliverables: Post-Launch Report, Lessons Learned Documentation

## 4. Team Roles and Responsibilities
- **Project Manager**: Oversees the project, ensures deadlines are met, handles stakeholder communication.
- **Product Owner**: Defines feature requirements, prioritizes backlog, liaises with stakeholders.
- **Frontend Developer**: Designs and implements the frontend of the feature.
- **Backend Developer**: Develops the backend logic and database interactions.
- **QA Engineer**: Conducts testing, creates test cases, reports bugs.
- **UI/UX Designer**: Creates wireframes and design mockups.

## 5. Timeline and Schedule
| Phase | Start Date | End Date | Key Activities |
|-------|------------|----------|----------------|
| Initiation | [Start Date] | [End Date] | Project setup, initial meetings |
| Design | [Start Date] | [End Date] | Wireframing, design mockups, user flow |
| Development | [Start Date] | [End Date] | Coding, integration, initial testing |
| Testing | [Start Date] | [End Date] | QA testing, bug fixing, UAT |
| Launch Preparation | [Start Date] | [End Date] | Final checks, deployment preparation |
| Post-Launch Review | [Start Date] | [End Date] | Performance review, feedback collection |

## 6. Risk Management
- **Identified Risks**:
  - Delays in design approvals
  - Integration issues with existing systems
  - Unforeseen bugs in testing
- **Mitigation Strategies**:
  - Regular check-ins with stakeholders for timely approvals.
  - Early integration with existing systems to identify issues early.
  - Comprehensive testing plan to catch bugs early.

## 7. Communication Plan
- **Stakeholder Meetings**: Weekly meetings with key stakeholders to provide updates.
- **Team Meetings**: Daily stand-up meetings to track progress and address issues.
- **Reporting**: Bi-weekly progress reports to the project sponsor.
- **Tools**: Use of project management tools (e.g., Jira, Trello) for task tracking and updates.

## 8. Quality Assurance
- **Testing Strategy**: 
  - Unit Testing: Developers to write and execute unit tests.
  - Integration Testing: Ensure components work together.
  - User Acceptance Testing (UAT): Conducted with a select group of end-users.
- **Bug Tracking**: Use Jira or similar tools to log and track bugs.

## 9. Budget and Resources
- **Budget**: Outline the budget including development costs, testing, and tools.
- **Resources**: List resources needed including software licenses, hardware, and third-party services.

## 10. Approval and Sign-off
- **Project Sponsor**: [Name of Sponsor]
- **Approval Process**: Approval required from the project sponsor at each milestone.

---

By adhering to this project plan, the team should be able to streamline the development process and ensure a timely and successful launch of the new app feature.

--------

The above is great but its too long, and you see it keeps going on more. As mentioned this is for Executives, do we want to keep it this long?


Lets now give some guiding questions for model to come up with a draft but first do the thinking


### With Guided Chain of Thought

In [9]:
guided_cot = """You are a project manager for a small software development team tasked with launching a new app feature. 
You want to streamline the development process and ensure timely delivery.	
Your task is to draft a project plan. 

But first do some thinking on how you want to structure and go through below questions before starting the draft
Please follow these steps:
1. Think about who the audience is (this is for CEOs, CTOs and other executives)
2. Think about what to start with
1. Think about what Challenges you want to solve with this app
2. Think about the Tasks that will be needed to be completed
3. Create Milestones
4. Monitor Progress and Optimize
Explain all your thinking in <thinking></thinking> XML Tags and then write the final copy of project plan for executives in <project_plan></project_plan> XML Tag.

Output Schema:
<thinking>
( thoughts to above questions)
</thinking>
<project_plan>
( project plan)
</project_plan>
"""
messages = [
        {"role": "user", "content": [{"text": guided_cot}]}
]

model_response, content_text = call_nova(LITE_MODEL_ID,
                                         messages, max_tokens=2048)
print("\n[Response Content Text]")
print_output("--------")
print(content_text)
print_output("--------")


[Response Content Text]


--------

<thinking>
Considering the audience for this project plan is the executive team, it's important to structure the document to be concise, clear, and focused on strategic value, high-level objectives, and potential challenges. The plan should highlight the purpose of the new app feature, the challenges it aims to solve, the key tasks, and milestones, as well as the approach to monitoring progress and ensuring optimization. Starting with a clear statement of purpose and challenges addresses the "why" behind the project, which is crucial for executive buy-in and understanding. Outlining tasks and milestones provides a roadmap for the project's execution, while the approach to monitoring and optimizing ensures the project remains on track and adapts to any unforeseen challenges.

The first step is to identify the challenges we aim to solve with this app feature. This could include enhancing user engagement, improving the app's functionality, or introducing new features to stay competitive i

--------

### Next Step

Lets move to creating a RAG system using  Bedrock Knowledge Bases